In [2]:
import getpass
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass

os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key: "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "default"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"
if "TAVILY_API_KEY" not in os.environ: 
    os.environ["TAVILY_API_KEY"] = getpass.getpass(
        prompt='Enter your Tavily API key: '
    )
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass(
        prompt="Enter your OpenAI API key: "
    )

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://weathershogun.com/weather/usa/ca/san-francisco/480/may/2025-05-17', 'content': 'San Francisco, California Weather: Saturday, May 17, 2025. Cloudy weather, overcast skies with clouds. Day 64°. Night 52°. Precipitation 1 %.'}, {'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1747440067, 'localtime': '2025-05-16 17:01'}, 'current': {'last_updated_epoch': 1747440000, 'last_updated': '2025-05-16 17:00', 'temp_c': 15.6, 'temp_f': 60.1, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 14.1, 'wind_kph': 22.7, 'wind_degree': 272, 'wind_dir': 'W', 'pressure_mb': 1014.0, 'pressure_in': 29.93, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 64, 'cloud': 75, 'feelslike_c': 15.6, 'feelslike_f': 60.1, 'windchill

In [4]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage

model = init_chat_model("gpt-4o-mini", model_provider="openai")

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [5]:
#bind tools to model 
model_with_tools = model.bind_tools(tools)

In [6]:
# call the model, print out the tools called 
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [10]:
# use another prompt so the model request to use tool
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather'}, 'id': 'call_3PfoXUXty5Zq6fS4FOjqxaM9', 'type': 'tool_call'}]


In [11]:
# Using langgraph for calling agent 
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools) # we pass in model, not model-with-tool, because create_react_agent will call bind_tools() under the hood 

In [12]:
# calling agent without a tool first 
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='895acb33-596e-4785-8550-b47dc4b2cb07'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 81, 'total_tokens': 91, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_54eb4bd693', 'id': 'chatcmpl-BXzrdvzcV9uAYUjO5fCNwau37ZJZd', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--af522407-d150-4ddb-a5d3-111bb4e5acbb-0', usage_metadata={'input_tokens': 81, 'output_tokens': 10, 'total_tokens': 91, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

In [13]:
# Now we try to invoke a tool 
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='1d1b0518-30f2-4933-b333-6725c3526cae'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_tX4ULi7f6mQB1kvSw6EQwpL5', 'function': {'arguments': '{"query":"San Francisco current weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 86, 'total_tokens': 107, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_54eb4bd693', 'id': 'chatcmpl-BXzsCfDjQkhxcilJ7JTI80n6taHBr', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--e4c199e4-45e4-4bd1-b141-9ee45103356f-0', tool_calls=[{'name': 'tavily_search_results_json', 'a

In [14]:
# We can stream back message as the tokens return 
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather in sf?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_1k5Exq2TwL0DL4pC1naaP8Wm)
 Call ID: call_1k5Exq2TwL0DL4pC1naaP8Wm
  Args:
    query: current weather in San Francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://weathershogun.com/weather/usa/ca/san-francisco/480/may/2025-05-17", "content": "San Francisco, California Weather: Saturday, May 17, 2025. Cloudy weather, overcast skies with clouds. Day 64°. Night 52°. Precipitation 1 %."}, {"url": "https://www.easeweather.com/north-america/united-states/california/city-and-county-of-san-francisco/san-francisco/may", "content": "Weather in San Francisco for May 2025. Your guide to San Francisco weather ... May 17. Partly cloudy. 57° /51°, 0 in, 0. May 18. Sunny. 60"}]
======

In [15]:
# We can also stream back tokens 
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The| current| weather| in| San| Francisco| shows| cloudy| weather| with| over|cast| skies|.| The| temperature| is| around| |64|°F| during| the| day| and| drops| to| about| |52|°F| at| night|,| with| a| low| chance| of| precipitation|.| 

|For| more| detailed| updates|,| you| can| check| the| sources| [|here|](|https|://|we|athers|hog|un|.com|/weather|/|usa|/|ca|/s|an|-fr|anc|isco|/|480|/m|ay|/|202|5|-|05|-|17|)| or| [|here|](|https|://|www|.ease|weather|.com|/n|orth|-amer|ica|/un|ited|-st|ates|/cal|ifornia|/c|ity|-and|-c|ounty|-of|-san|-fr|anc|isco|/s|an|-fr|anc|isco|/m|ay|).|

In [16]:
# We can also add checkpoint and thread id, so the agent knows where to return from 
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [17]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [18]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 83, 'total_tokens': 94, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_54eb4bd693', 'id': 'chatcmpl-BXzxmfEP8CElSwWH4oML8Kxo6JAtz', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c94d66bc-b70b-4653-90b1-ba2220c62bcb-0', usage_metadata={'input_tokens': 83, 'output_tokens': 11, 'total_tokens': 94, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----


In [19]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob! How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 106, 'total_tokens': 119, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_54eb4bd693', 'id': 'chatcmpl-BXzy4bYbDO16QJx3s4Fo43OH9ZcPa', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--9de6f0b0-01b0-4dbd-ac8c-39ea0be35f2d-0', usage_metadata={'input_tokens': 106, 'output_tokens': 13, 'total_tokens': 119, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----


In [20]:
# If you want to start a new conversation, you can change to a new thread id 
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I don't have access to personal data about users unless it has been shared with me in the course of our conversation. If you tell me your name, I'd be happy to remember it for the duration of our chat!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 84, 'total_tokens': 128, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_54eb4bd693', 'id': 'chatcmpl-BXzyegmpOQwqYtG4JC18smxqj5Ja7', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--d565cd5f-d798-474c-911b-94ab6a3ebc58-0', usage_metadata={'input_tokens': 84, 'output_tokens': 44, 'total_tokens': 128, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'au